This file is for merging the yellow, green, fhv and hvfhv taxi data

In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os

In [2]:
# Get list of each filename
yellow_files = [filename for filename in os.listdir('.') if filename.startswith("yellow") and filename.endswith(".parquet")]
green_files = [filename for filename in os.listdir('.') if filename.startswith("green") and filename.endswith(".parquet")] 
fhv_files = [filename for filename in os.listdir('.') if filename.startswith("fhv_") and filename.endswith(".parquet")] #Need underscore to ensure it doesn't pick up fhvhv
fhvhv_files = [filename for filename in os.listdir('.') if filename.startswith("fhvhv") and filename.endswith(".parquet")]

In [7]:
def merge_files(filenames, output_name):
    # merge multiple files into one parquet file
    schema = pq.ParquetFile(filenames[0]).schema_arrow
    with pq.ParquetWriter(output_name, schema=schema) as writer:
        for file in filenames:
            table = pq.read_table(file, schema=schema)
            writer.write_table(table)
            print("Read", file)

In [ ]:
def merge_files_green(filenames, output_name):
    # For green taxi data need to take the most recent schema or it will crash
    schema = pq.ParquetFile(filenames[-1]).schema_arrow
    with pq.ParquetWriter(output_name, schema=schema) as writer:
        for file in filenames:
            table = pq.read_table(file, schema=schema)
            writer.write_table(table)
            print("Read", file)

In [4]:
print(pq.ParquetFile(fhv_files[0]).schema_arrow)

dispatching_base_num: string
pickup_datetime: timestamp[us]
dropOff_datetime: timestamp[us]
PUlocationID: double
DOlocationID: double
SR_Flag: null
Affiliated_base_number: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1000


In [9]:
files_to_merge = [fhv_files] # [yellow_files, green_files, fhv_files, fhvhv_files]
output_names = ["all_fhv_tripdata.parquet"] # ["all_yellow_tripdata.parquet", "all_green_tripdata.parquet", "all_fhv_tripdata.parquet", "all_fhvhv_tripdata.parquet"] # start name with all to ensure 
# NOTE: FHVHV files have to be done separately (below) due to malloc issues
for filenames, output_name in zip(files_to_merge, output_names):
    merge_files(filenames, output_name)
    print(output_name, "merged")

Read fhv_tripdata_2021-01.parquet
Read fhv_tripdata_2021-02.parquet
Read fhv_tripdata_2021-03.parquet
Read fhv_tripdata_2021-04.parquet
Read fhv_tripdata_2021-05.parquet
Read fhv_tripdata_2021-06.parquet
Read fhv_tripdata_2021-07.parquet
Read fhv_tripdata_2021-08.parquet
Read fhv_tripdata_2021-09.parquet
Read fhv_tripdata_2021-10.parquet
Read fhv_tripdata_2021-11.parquet
Read fhv_tripdata_2021-12.parquet
Read fhv_tripdata_2022-01.parquet
Read fhv_tripdata_2022-02.parquet
Read fhv_tripdata_2022-03.parquet
Read fhv_tripdata_2022-04.parquet
Read fhv_tripdata_2022-05.parquet
Read fhv_tripdata_2022-06.parquet
Read fhv_tripdata_2022-07.parquet
Read fhv_tripdata_2022-08.parquet
Read fhv_tripdata_2022-09.parquet
Read fhv_tripdata_2022-10.parquet
Read fhv_tripdata_2022-11.parquet
Read fhv_tripdata_2022-12.parquet
Read fhv_tripdata_2023-01.parquet
Read fhv_tripdata_2023-02.parquet
Read fhv_tripdata_2023-03.parquet
Read fhv_tripdata_2023-04.parquet
Read fhv_tripdata_2023-05.parquet
Read fhv_tripd

In [6]:
print(pq.ParquetFile(fhvhv_files[-1]).schema_arrow)

hvfhs_license_num: large_string
dispatching_base_num: large_string
originating_base_num: large_string
request_datetime: timestamp[us]
on_scene_datetime: timestamp[us]
pickup_datetime: timestamp[us]
dropoff_datetime: timestamp[us]
PULocationID: int32
DOLocationID: int32
trip_miles: double
trip_time: int64
base_passenger_fare: double
tolls: double
bcf: double
sales_tax: double
congestion_surcharge: double
airport_fee: double
tips: double
driver_pay: double
shared_request_flag: large_string
shared_match_flag: large_string
access_a_ride_flag: large_string
wav_request_flag: large_string
wav_match_flag: large_string


In [6]:
table = pq.read_table("fhvhv_tripdata_2023-01.parquet")
table = table.drop(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'])
filtered_schema = table.schema
df = table.to_pandas()
df.columns

Index(['pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID'], dtype='object')

In [7]:
# merge multiple files into one parquet file
schema = pq.ParquetFile(fhvhv_files[0]).schema_arrow
with pq.ParquetWriter("all_fhvhv_tripdata.parquet", schema=filtered_schema) as writer:
    for file in fhvhv_files:
        table = pq.read_table(file)
        table = table.drop(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
        'request_datetime', 'on_scene_datetime', 'trip_miles',
        'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
        'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
        'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
        'wav_request_flag', 'wav_match_flag'])
        table = table.cast(filtered_schema)
        writer.write_table(table)
        print("Read", file)

Read fhvhv_tripdata_2021-01.parquet
Read fhvhv_tripdata_2021-02.parquet
Read fhvhv_tripdata_2021-03.parquet
Read fhvhv_tripdata_2021-04.parquet
Read fhvhv_tripdata_2021-05.parquet
Read fhvhv_tripdata_2021-06.parquet
Read fhvhv_tripdata_2021-07.parquet
Read fhvhv_tripdata_2021-08.parquet
Read fhvhv_tripdata_2021-09.parquet
Read fhvhv_tripdata_2021-10.parquet
Read fhvhv_tripdata_2021-11.parquet
Read fhvhv_tripdata_2021-12.parquet
Read fhvhv_tripdata_2022-01.parquet
Read fhvhv_tripdata_2022-02.parquet
Read fhvhv_tripdata_2022-03.parquet
Read fhvhv_tripdata_2022-04.parquet
Read fhvhv_tripdata_2022-05.parquet
Read fhvhv_tripdata_2022-06.parquet
Read fhvhv_tripdata_2022-07.parquet
Read fhvhv_tripdata_2022-08.parquet
Read fhvhv_tripdata_2022-09.parquet
Read fhvhv_tripdata_2022-10.parquet
Read fhvhv_tripdata_2022-11.parquet
Read fhvhv_tripdata_2022-12.parquet
Read fhvhv_tripdata_2023-01.parquet
Read fhvhv_tripdata_2023-02.parquet
Read fhvhv_tripdata_2023-03.parquet
Read fhvhv_tripdata_2023-04.

In [9]:
# merge multiple files into one parquet file
schema = pq.ParquetFile(fhvhv_files[0]).schema_arrow
with pq.ParquetWriter("all_fhvhv_tripdata.parquet", schema=filtered_schema) as writer:

    table = pq.read_table("fhvhv_tripdata_2023-02.parquet")
    table = table.drop(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
    'request_datetime', 'on_scene_datetime', 'trip_miles',
    'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
    'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
    'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
    'wav_request_flag', 'wav_match_flag'])
    table = table.cast(filtered_schema)
    writer.write_table(table)
    print("Read", file)

Read fhvhv_tripdata_2024-05.parquet


In [10]:
for file in fhvhv_files:
    print(pq.ParquetFile(fhvhv_files[0]).schema_arrow == pq.ParquetFile(file).schema_arrow, file)

True fhvhv_tripdata_2021-01.parquet
True fhvhv_tripdata_2021-02.parquet
True fhvhv_tripdata_2021-03.parquet
True fhvhv_tripdata_2021-04.parquet
True fhvhv_tripdata_2021-05.parquet
True fhvhv_tripdata_2021-06.parquet
True fhvhv_tripdata_2021-07.parquet
True fhvhv_tripdata_2021-08.parquet
True fhvhv_tripdata_2021-09.parquet
True fhvhv_tripdata_2021-10.parquet
True fhvhv_tripdata_2021-11.parquet
True fhvhv_tripdata_2021-12.parquet
True fhvhv_tripdata_2022-01.parquet
True fhvhv_tripdata_2022-02.parquet
True fhvhv_tripdata_2022-03.parquet
True fhvhv_tripdata_2022-04.parquet
True fhvhv_tripdata_2022-05.parquet
True fhvhv_tripdata_2022-06.parquet
True fhvhv_tripdata_2022-07.parquet
True fhvhv_tripdata_2022-08.parquet
True fhvhv_tripdata_2022-09.parquet
True fhvhv_tripdata_2022-10.parquet
True fhvhv_tripdata_2022-11.parquet
True fhvhv_tripdata_2022-12.parquet
True fhvhv_tripdata_2023-01.parquet
False fhvhv_tripdata_2023-02.parquet
False fhvhv_tripdata_2023-03.parquet
False fhvhv_tripdata_2023-

# Sample of each table

In [11]:
table1 = pq.read_table('green_tripdata_2024-01.parquet')
table1.to_pandas()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2024-01-01 00:46:55,2024-01-01 00:58:25,N,1.0,236,239,1.0,1.98,12.80,1.0,0.5,3.61,0.0,NaN,1.0,21.66,1.0,1.0,2.75
1,2,2024-01-01 00:31:42,2024-01-01 00:52:34,N,1.0,65,170,5.0,6.54,30.30,1.0,0.5,7.11,0.0,NaN,1.0,42.66,1.0,1.0,2.75
2,2,2024-01-01 00:30:21,2024-01-01 00:49:23,N,1.0,74,262,1.0,3.08,19.80,1.0,0.5,3.00,0.0,NaN,1.0,28.05,1.0,1.0,2.75
3,1,2024-01-01 00:30:20,2024-01-01 00:42:12,N,1.0,74,116,1.0,2.40,14.20,1.0,1.5,0.00,0.0,NaN,1.0,16.70,2.0,1.0,0.00
4,2,2024-01-01 00:32:38,2024-01-01 00:43:37,N,1.0,74,243,1.0,5.14,22.60,1.0,0.5,6.28,0.0,NaN,1.0,31.38,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56546,2,2024-01-31 20:46:00,2024-01-31 20:55:00,None,NaN,33,25,NaN,0.00,11.58,0.0,0.0,3.14,0.0,NaN,1.0,15.72,NaN,NaN,NaN
56547,2,2024-01-31 21:06:00,2024-01-31 21:11:00,None,NaN,72,72,NaN,0.49,11.58,0.0,0.0,0.00,0.0,NaN,1.0,12.58,NaN,NaN,NaN
56548,2,2024-01-31 21:36:00,2024-01-31 21:40:00,None,NaN,72,72,NaN,0.52,11.58,0.0,0.0,2.52,0.0,NaN,1.0,15.10,NaN,NaN,NaN
56549,2,2024-01-31 22:45:00,2024-01-31 22:51:00,None,NaN,41,42,NaN,1.17,14.22,0.0,0.0,0.00,0.0,NaN,1.0,15.22,NaN,NaN,NaN


In [12]:
table2 = pq.read_table('fhv_tripdata_2024-01.parquet')
table2.to_pandas()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00053,2024-01-01 00:15:00,2024-01-01 02:13:00,NaN,NaN,NaN,B00014
1,B00111,2024-01-01 00:30:00,2024-01-01 02:37:00,NaN,NaN,NaN,B00111
2,B00112,2024-01-01 00:27:24,2024-01-01 01:12:05,NaN,14.0,NaN,B00112
3,B00112,2024-01-01 00:10:09,2024-01-01 00:25:39,NaN,133.0,NaN,B00112
4,B00112,2024-01-01 00:57:07,2024-01-01 01:05:04,NaN,14.0,NaN,B00112
...,...,...,...,...,...,...,...
1290111,B03492,2024-01-31 23:51:21,2024-01-31 23:57:09,NaN,14.0,NaN,B03492
1290112,B03492,2024-01-31 23:05:06,2024-01-31 23:20:13,NaN,21.0,NaN,B03492
1290113,B03492,2024-01-31 23:49:07,2024-02-01 00:16:25,NaN,14.0,NaN,B03492
1290114,B03505,2024-01-31 23:04:46,2024-01-31 23:18:22,NaN,76.0,NaN,B03505


In [13]:
table3 = pq.read_table('fhvhv_tripdata_2024-01.parquet')
table3.to_pandas()

In [ ]:
print(table.shape[1])
print(table1.shape[1])
print(table2.shape[1])
print(table3.shape[1])

19
20
7
24
